In [1]:
from tqdm import tqdm
import pickle

# facts = pickle.load(open('wikidata_text_facts.txt', 'rb'))
# facts = pickle.load(open('text_facts_v2.pkl', 'rb'))

In [2]:
def getEntsFromFacts(facts):
    e = set()
    for f in facts:
        e.add(f[0])
        e.add(f[2])
    return e

def readFile(filename):
    f = open(filename, 'r')
    lines = []
    for line in f:
        line = line.strip()
        if line != '':
            lines.append(line)
    f.close()
    return lines

def getRelsFromFacts(facts):
    r = set()
    for f in facts:
        r.add(f[1])
    return r

def filterByRelation(facts, rel, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[1] == rel:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def filterByHead(facts, head, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[0] == head:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def filterByTail(facts, tail, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[2] == tail:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def filterByEntity(facts, ent, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[0] == ent or f[2] == ent:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def printFact(f):
    s = "{head}, {rel}, {tail}, {t1}, {t2}"
    head=f[0]
    rel = f[1]
    tail = f[2]
    t1 = f[3][1:5]
    t2 = f[4][1:5]
    print(s.format(head=head, rel=rel, tail=tail, t1=t1, t2=t2))
    
def printFacts(facts):
    for f in facts:
        printFact(f)
        
def isEntityInFact(e, fact):
    if fact[0] == e or fact[2] == e:
        return True
    else:
        return False
    
        
def writeFactsToFile(filename, facts):
    f = open(filename, 'w')
    for fact in facts:
        line = '\t'.join(fact)
        f.write(line + '\n')
    f.close()
    
def readFactsFromFile(filename):
    f = open(filename, 'r')
    facts = []
    for line in f:
        fact = line.strip().split('\t')
        facts.append(fact)
    f.close()
    return facts

def createWikidataIdentifierToTextDict(entities, relations):
    ent_rel_both = {}
    ent_rel_both['Q'] = entities
    ent_rel_both['P'] = relations
    wikidata_id2name = {}
    count = 0
    for prefix, lines in ent_rel_both.items():
        for line in lines:
            try:
                line = line.split('\t')
                id = prefix + line[1] #different from one above
                name = line[2]
                wikidata_id2name[id] = name
            except:
                id = prefix + line[1]
                name = ''
                wikidata_id2name[id] = name
    return wikidata_id2name

def convertFactToText(fact, wikidata_identifier_to_text):
    f = fact.copy()
    for i in range(3):
        f[i] = wikidata_identifier_to_text[fact[i]]
    return f

In [3]:
facts = readFactsFromFile('..data/temporal_big/full.txt')

In [4]:
r = getRelsFromFacts(facts)
e = getEntsFromFacts(facts)

In [5]:
e_count_dict = {}
r_count_dict = {}
head_count = {}
tail_count = {}
for ent in e:
    e_count_dict[ent] = 0
    head_count[ent] = 0
    tail_count[ent] = 0
for rel in r:
    r_count_dict[rel] = 0
for f in facts:
    e1 = f[0]
    e2 = f[2]
    rel = f[1]
    e_count_dict[e1] += 1
    e_count_dict[e2] += 1
    r_count_dict[rel] += 1
    head_count[e1] += 1
    tail_count[e2] += 1

In [6]:
x = r_count_dict
sorted_r = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

x = e_count_dict
sorted_e = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

x = head_count
sorted_head = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

x = tail_count
sorted_tail = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
for key, value in sorted_r.items():
    print(key, '|', value, 'facts', round(value/len(facts)*100, 2), '%' )

In [8]:
rel = 'P166'
awardFacts = filterByRelation(facts, rel, -1)

In [9]:
def splitTimeNoNewTimeStamps(facts, timestamps):
    # don't introduce new timestamps!
    new_facts = []
    for f in facts:
        e1 = f[0]
        r = f[1]
        e2 = f[2]
        t1 = int(f[3])
        t2 = int(f[4])
        for t in range(t1, t2+1):
            if t in timestamps:
                nf = [e1, r, e2, t]
                new_facts.append(nf)
    return new_facts

def splitTime(facts):
    # don't introduce new timestamps!
    new_facts = []
    for f in facts:
        e1 = f[0]
        r = f[1]
        e2 = f[2]
        t1 = int(f[3])
        t2 = int(f[4])
        for t in range(t1, t2+1):
            nf = [e1, r, e2, t]
            new_facts.append(nf)
    return new_facts



In [10]:
awardFactsSingle= splitTime(awardFacts)

In [11]:
# simple questions, time answer
def genTimeSimple1(facts, base_fact):
    head = base_fact[0]
    tail = base_fact[2]
    template = "When did {head} receive {tail}?"
    answers = set()
    for f in facts:
        if f[0] == head and f[2] == tail:
            answers.add(f[3])
    question = template.format(head=head, tail=tail)
    answer_type = 'time'
    entities = set([head, tail])
    times = set()
    relations = set(['P166'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

def genEntitySimple1(facts, head, time):
    template = "Which award did {head} receive in {time}?"
    answers = set()
    for f in facts:
        if f[0] == head and f[3] == time:
            answers.add(f[2])
    question = template.format(head=head, time=time)
    answer_type = 'entity'
    entities = set([head])
    times = set([time])
    relations = set(['P166'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output
        

# complex question, entity answer
def genEntityComplex1(facts, head, first=True):
    # first/last
    template = "Which was the {adj} award that {head} received?"
    minTime = 9999
    minTeam = ""
    maxTime = -1
    maxTeam = ""
    answers = set()
    #go through all the facts and find the min[max] time
    for f in facts:
        if f[0] == head:
            time = f[3]
            if time < minTime:
                minTime = time
            if time > maxTime:
                maxTime = time
    #go through all the facts and this time fill answer set if the fact's time matches with min[max] time
    for f in facts:
        if f[0] == head:
            time = f[3]
            if first==True:
                if time==minTime:
                    answers.add(f[2])
            else:
                if time==maxTime:
                    answers.add(f[2])
    if first==True:
        question = template.format(head=head, adj='first')
    else:
        question = template.format(head=head, adj='last')
    answer_type = 'entity'
    entities = set([head])
    times = set()
    relations = set(['P166'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output
        

def getFactWithMaximumTime(facts, head = '', tail = ''):
    maxTime = -1
    fact = facts[0]
    for f in facts:
        time = f[-1]
        if head != '':
            if f[0] != head:
                continue
        if tail != '':
            if f[2] != tail:
                continue
        if time > maxTime:
            maxTime = time
            fact = f
    return fact

def getFactWithMinimumTime(facts, head = '', tail = ''):
    minTime = 9999
    fact = facts[0]
    for f in facts:
        time = f[-1]
        if head != '':
            if f[0] != head:
                continue
        if tail != '':
            if f[2] != tail:
                continue
        if time < minTime:
            minTime = time
            fact = f
    return fact


def genEntityComplex2(facts, head, tail, after=True):
    # before/after
    template = "Who received the {tail} {type} {head}?"
    # before: find argmin(T) (head, r, tail, T)
    # then, find argmax(T')(head, r, tail', T'), T' < T and tail' != tail
    # tail' is the answer
    answers = set()
    if after == False: # if "before"
        base_fact = getFactWithMinimumTime(facts, head = head, tail = tail)
        time = base_fact[-1]
        maxTime = -1
        for f in facts:
            time2 = f[-1]
            if time2 >= time: 
                continue
            if f[0] == head: #ok, because with the same head, fact with min time is taken 
                continue
            if f[2] != tail:
                continue
            if time2 > maxTime:
                maxTime = time2
        answers = set()
        for f in facts:
            time2 = f[-1]
            if time2 == maxTime and f[2]==tail:
                answers.add(f[0])
        question = template.format(head=head, type="before", tail=tail)
    else:
        # after: find argmax(T) (head, r, tail, T)
        # then, find argmin(T') (head, r, tail', T'), T' > T and tail' != tail
        # tail' is the answer
        base_fact = getFactWithMaximumTime(facts, head = head, tail = tail)
        time = base_fact[-1]
        minTime = 9999
        for f in facts:
            time2 = f[-1]
            if time2 <= time:
                continue
            if f[0] == head:
                continue
            if f[2] != tail:
                continue
            if time2 < minTime:
                minTime = time2
        answers = set()
        for f in facts:
            time2 = f[-1]
            if time2 == minTime and f[2]==tail:
                answers.add(f[0])
        question = template.format(head=head, type="after", tail=tail)
    #following is necessary in case there is no answer to the question
    if len(answers)==0:
        return None
    answer_type = 'entity'
    entities = set([head, tail])
    times = set()
    relations = set(['P166'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output
        


def areFactsSame(f1, f2):
    flag = True
    for i in range(3):
        if f1[i] != f2[i]:
            flag = False
            break
    return flag

In [17]:
# len(events_facts)
import pickle as pkl
def openFileAsDict(filename):
    f = open(filename, 'r')
    out = {}
    for line in f:
        line = line[:-1].split('\t') # can't strip() since name can be whitespace
        out[line[0]] = line[1]
    return out
ent2name = openFileAsDict( ../data/wikidata_big/kg/wd_id2entity_text.txt')
rel2name = openFileAsDict( '../data/wikidata_big/kg/wd_id2relation_text.txt')
name2ent={i:j for (j,i) in ent2name.items()}
name2rel={i:j for (j,i) in rel2name.items()}

ent2id=pkl.load(open("../data/wikidata_big/kg/tkbc_processed_data/wikidata_big/ent_id","rb"))
rel2id=pkl.load(open("../data/wikidata_big/kg/tkbc_processed_data/wikidata_big/rel_id","rb"))
id2ent={i:j for (j,i) in ent2id.items()}
id2rel={i:j for (j,i) in rel2id.items()}
id2relname={i:rel2name[id2rel[i]] for i in id2rel}
id2entname={i:ent2name[id2ent[i]] for i in id2ent}
def get_facts_from_entname(entname):
    idx=name2ent[entname]
    facts_=[]
    for fact in events_facts:
        if (fact[0]==idx or fact[2]==idx):
            fact=(ent2name[fact[0]],rel2name[fact[1]],ent2name[fact[2]],fact[3],fact[4])
            facts_.append(fact)
            print(fact)
    return facts_
def get_facts_from_entid(idx):
    facts_=[]
    for fact in awardFacts:
        if (fact[0]==idx or fact[2]==idx):
            fact=(ent2name[fact[0]],rel2name[fact[1]],ent2name[fact[2]],fact[3],fact[4])
            facts_.append(fact)
#             print(fact)
    return facts_


In [20]:
get_facts_from_entid("Q57464"),get_facts_from_entid("Q981023"),get_facts_from_entid("Q460431")

([('Abdullah II of Jordan',
   'award received',
   'Grand cross of the Order of the White Lion',
   '2015',
   '2015'),
  ('Abdullah II of Jordan',
   'award received',
   'Grand Cross special issue of the Order of Merit of the Federal Republic of Germany, special issue',
   '2002',
   '2002')],
 [('Josef František',
   'award received',
   'Grand cross of the Order of the White Lion',
   '2015',
   '2015')],
 [('Nicholas Winton',
   'award received',
   'Grand cross of the Order of the White Lion',
   '2014',
   '2014'),
  ('Nicholas Winton', 'award received', 'Knight Bachelor', '2002', '2002')])

In [23]:
id = 1
f = awardFactsSingle[id]
genEntityComplex2(awardFactsSingle, f[0], f[2])

{'question': 'Who received the Q20966518 after Q460431?',
 'answers': {'Q57464', 'Q981023'},
 'answer_type': 'entity',
 'template': 'Who received the {tail} {type} {head}?',
 'entities': {'Q20966518', 'Q460431'},
 'times': set(),
 'relations': {'P166'}}

In [24]:
get_facts_from_entid("Q460431")

[('Nicholas Winton',
  'award received',
  'Grand cross of the Order of the White Lion',
  '2014',
  '2014'),
 ('Nicholas Winton', 'award received', 'Knight Bachelor', '2002', '2002')]

In [21]:
id = 1
f = awardFactsSingle[id]
genTimeSimple1(awardFactsSingle, f)

{'question': 'When did Q460431 receive Q20966518?',
 'answers': {2014},
 'answer_type': 'time',
 'template': 'When did {head} receive {tail}?',
 'entities': {'Q20966518', 'Q460431'},
 'times': set(),
 'relations': {'P166'}}

In [25]:
genEntitySimple1(awardFactsSingle, f[0], f[-1])

{'question': 'Which award did Q460431 receive in 2014?',
 'answers': {'Q20966518'},
 'answer_type': 'entity',
 'template': 'Which award did {head} receive in {time}?',
 'entities': {'Q460431'},
 'times': {2014},
 'relations': {'P166'}}

In [ ]:
get_facts_from_entid("Q833163")

In [26]:
genEntityComplex1(awardFactsSingle, f[0], first=True)

{'question': 'Which was the first award that Q460431 received?',
 'answers': {'Q833163'},
 'answer_type': 'entity',
 'template': 'Which was the {adj} award that {head} received?',
 'entities': {'Q460431'},
 'times': set(),
 'relations': {'P166'}}

In [29]:
id = 50
f = awardFactsSingle[id]
genEntityComplex2(awardFactsSingle, f[0], f[2])

{'question': 'Who received the Q19573074 after Q1299823?',
 'answers': {'Q182727'},
 'answer_type': 'entity',
 'template': 'Who received the {tail} {type} {head}?',
 'entities': {'Q1299823', 'Q19573074'},
 'times': set(),
 'relations': {'P166'}}

In [31]:
import random
random.seed(1000)
def makeQuestions(awardFactsSingle, f):
    questions = []
    questions.append(genTimeSimple1(awardFactsSingle, f))
    questions.append(genEntitySimple1(awardFactsSingle, f[0], f[-1]))
    questions.append(genEntityComplex1(awardFactsSingle, f[0], first=random.choice([True, False])))
    entityComplex2ques=genEntityComplex2(awardFactsSingle, f[0], f[2], after=random.choice([True, False]))
    if entityComplex2ques is not None:
        questions.append(entityComplex2ques)
    else:
        print("None detected")
        
        
    return questions



In [35]:
import pickle
dataset_split = 'train'
split_entities = set(pickle.load(open(dataset_split + '_ents.pickle', 'rb')))
split_facts = []
my_facts = awardFactsSingle
for f in my_facts:
    if f[0] in split_entities and f[2] in split_entities:
        split_facts.append(f)
len(split_facts)

38235

In [36]:
# how to get number of questions?
# 1. set total questions to 300k
# 2. get number of facts with this relation
# 3. get how many questions u want of this relation as fraction
# 4. choose 10% of that number for test, 90% for train
max_dataset_questions = 300000
relation_name = my_facts[0][1]
num_relation_facts = len(filterByRelation(facts, relation_name, -1))
# fraction of questions need to be decided based on fraction of question relation facts
# not all facts!
question_relation_list = ['P39', 'P166', 'P108', 'P54', 'P26']
num_all_question_relation_facts = sum([len(filterByRelation(facts, x, -1)) for x in question_relation_list])
num_questions_for_this_relation = int(max_dataset_questions * num_relation_facts/num_all_question_relation_facts)
split_ratios = {'test': 0.1, 'train': 0.9}
num_questions = int(split_ratios[dataset_split] * num_questions_for_this_relation)
num_questions

87705

In [ ]:
import random
data = []
# num_questions has been calculated above
# num_questions = 100000

questions_set = set()

pbar = tqdm(range(len(split_facts)))
random.shuffle(split_facts)
for i in range(len(split_facts)):
#     f = random.choice(positionFactsSingle)
    f = split_facts[i]
    pbar.set_description("Num questions %d, i %d" % (len(data), i))
    questions = makeQuestions(my_facts, f)
    for q in questions:
        if len(q['answers']) > 0 and q['question'] not in questions_set:
            data.append(q)
            questions_set.add(q['question'])
    if len(data) >= num_questions:
        break

In [40]:
def getQuestionTypeDistribution(data):
    type_dict = {}
    for d in data:
        template = d['template']
        if template not in type_dict:
            type_dict[template] = 1
        else:
            type_dict[template] += 1
    return type_dict
getQuestionTypeDistribution(data)

{'When did {head} receive {tail}?': 23809,
 'Which award did {head} receive in {time}?': 23560,
 'Which was the {adj} award that {head} received?': 17646,
 'Who received the {tail} {type} {head}?': 22691}

In [41]:
import pickle
filename = 'data/questions/{split}_questions_award_received_big.pickle'.format(
            split=dataset_split)
pickle.dump(data, open(filename, "wb"))

In [ ]:
import random

for i in tqdm(range(10000)):
    id = random.randint(0, len(sportsFactsSingle))
    f = sportsFactsSingle[id]
    try:
        data += makeQuestions(sportsFactsSingle, f)
    except:
        continue

 71%|███████   | 7098/10000 [2:30:52<1:02:43,  1.30s/it]

In [ ]:
data[10:]

In [ ]:
def writeQuestions(filename, data):
    f = open(filename, 'w')
    for d in data:
        answers_str = []
        for ans in d[1]:
            answers_str.append(str(ans))
        if answers_str == []:
            continue
        line = d[0] + '\t' + '|'.join(answers_str)
        f.write(line + '\n')
    f.close()

In [ ]:
writeQuestions('questions_member_of_sports_team.txt', data)